# Calculate connections

For most network analysis we do not consider actual geographic connections - this notebook focuses on how to calculate those direct connections between points and collect the geospatial information

In [1]:
import os, sys, time
import rasterio

import networkx as nx
import geopandas as gpd
import pandas as pd

from shapely.geometry import MultiLineString

In [2]:
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append("../")
import GOSTnets as gn

In [4]:
# Define input data
pth = "./"
# Read in cleaned pickle from earlier analysis and convert to time
G = nx.read_gpickle(os.path.join(pth, 'tutorial_outputs', r'iceland_network_clean.pickle'))
G_time = gn.convert_network_to_time(G, distance_tag = 'length', road_col = 'infra_type', factor = 1000)
# Define origins and destinations files
rek_grid_file = os.path.join(pth, "tutorial_data", "rek_grid.shp")
rek_pop_grid_file = rek_grid_file.replace(".shp", "_pop.shp")
churches_file = os.path.join(pth, "tutorial_data", "churches.shp")
# Read in origins and destinations files
rek_grid = gpd.read_file(rek_pop_grid_file)
in_churches = gpd.read_file(churches_file)
in_churches = in_churches.to_crs(rek_grid.crs)

In [6]:
# calculate the origins and destinations by snapping to the road network
origins_df = gn.pandana_snap_c(G_time, rek_grid, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
origins = list(set(origins_df['NN']))
destinations_df = gn.pandana_snap_c(G_time, in_churches, source_crs='epsg:4326', target_crs='epsg:4326',add_dist_to_node_col = True)
destinations = list(set(destinations_df['NN']))

In [7]:
nodes_gdf = gn.node_gdf_from_graph(G_time)
edges_gdf = gn.edge_gdf_from_graph(G_time)

In [9]:
obj_nodes = nx.shortest_path(G_time, source=origins[0], target=destinations[0], weight="time")
print(origins[0])
print(destinations[0])
obj_nodes # this is a list of the nodes that connected make the shortest path from the origin to the destination

3
15556


[3,
 7108,
 1796,
 343,
 'new_obj_3',
 7250,
 15803,
 2662,
 1644,
 9078,
 5995,
 5623,
 403,
 2928,
 11740,
 9270,
 7164,
 2734,
 4478,
 10552,
 3703,
 15165,
 5550,
 3054,
 11825,
 9667,
 13777,
 14560,
 6195,
 4675,
 9318,
 'new_obj_267',
 'new_obj_114',
 6351,
 215,
 'new_obj_144',
 'new_obj_244',
 13246,
 'new_obj_270',
 'new_obj_261',
 9247,
 2541,
 'new_obj_108',
 7513,
 11169,
 2162,
 'new_obj_36',
 8509,
 14705,
 'new_obj_69',
 'new_obj_55',
 'new_obj_68',
 'new_obj_65',
 'new_obj_109',
 10424,
 6678,
 15556]

## Calculate line strings connecting all origins to all destinations

In [11]:
all_res = []
all_connections = []
oIdx = 0
for org in origins:
    oIdx = oIdx + 1
    print(f'{oIdx} of {len(origins)}')
    for dest in destinations:
        obj_nodes = nx.shortest_path(G_time, source=org, target=dest, weight="time")
        all_edges = []
        for idx in range(0, len(obj_nodes) - 1):
            start_node = obj_nodes[idx]
            end_node = obj_nodes[idx + 1]
            cur_edge = edges_gdf.loc[(edges_gdf['stnode'] == start_node) & (edges_gdf['endnode'] == end_node), 'geometry'].iloc[0]
            all_edges.append(cur_edge)
            all_connections.append([start_node, end_node, cur_edge])
        all_res.append([org, dest, MultiLineString(all_edges)])

1 of 409
2 of 409
3 of 409
4 of 409
5 of 409
6 of 409
7 of 409
8 of 409
9 of 409
10 of 409
11 of 409
12 of 409
13 of 409
14 of 409
15 of 409
16 of 409
17 of 409
18 of 409
19 of 409
20 of 409
21 of 409
22 of 409
23 of 409
24 of 409
25 of 409
26 of 409
27 of 409
28 of 409
29 of 409
30 of 409
31 of 409
32 of 409
33 of 409
34 of 409
35 of 409
36 of 409
37 of 409
38 of 409
39 of 409
40 of 409
41 of 409
42 of 409
43 of 409
44 of 409
45 of 409
46 of 409
47 of 409
48 of 409
49 of 409
50 of 409
51 of 409
52 of 409
53 of 409
54 of 409
55 of 409
56 of 409
57 of 409
58 of 409
59 of 409
60 of 409
61 of 409
62 of 409
63 of 409
64 of 409
65 of 409
66 of 409
67 of 409
68 of 409
69 of 409
70 of 409
71 of 409
72 of 409
73 of 409
74 of 409
75 of 409
76 of 409
77 of 409
78 of 409
79 of 409
80 of 409
81 of 409
82 of 409
83 of 409
84 of 409
85 of 409
86 of 409
87 of 409
88 of 409
89 of 409
90 of 409
91 of 409
92 of 409
93 of 409
94 of 409
95 of 409
96 of 409
97 of 409
98 of 409
99 of 409
100 of 409
101 of 4

In [12]:
# Write all connections to file
all_results = pd.DataFrame(all_res, columns=['O','D','geometry'])
all_results.to_csv(os.path.join(pth, "tutorial_data","all_OD_links.csv"))

In [34]:
# Tabulate usage of individual links and write to file
all_conn = pd.DataFrame(all_connections, columns=['start','node','geometry'])
all_connections_count = pd.DataFrame(all_conn.groupby(['start','node']).count())
all_connections_count.reset_index(inplace=True)
all_connections_first = pd.DataFrame(all_conn.groupby(['start','node']).first())
all_connections_first.reset_index(inplace=True)
all_connections_first['count'] = all_connections_count['geometry']

In [44]:
all_connections_first.to_csv(os.path.join(pth, "tutorial_data","OD_links_usage.csv"))